# Notes Module
We are going to implement a attention mechanism similar to RETAIN.

alpha will be a scalar attention which will measure the importance of the Notes of particular day in the final output.
beta will be a vector of attention weights which will measure the importance of each notes embeding value (feature) in the final output.

In [ ]:
class NotesRNN(nn.Module):
    
    def __init__(self, notes_emb_size=128, input_notes_emb_size=200):
        super().__init__()
        
        self.emb_size = notes_emb_size
        self.RNN = nn.GRU(input_size = input_notes_emb_size, hidden_size = notes_emb_size, batch_first = True)
        self.fc1 = nn.Linear(notes_emb_size, notes_emb_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        #self.fc2 = nn.Linear(notes_emb_size,128)
        #self.sig = nn.Sigmoid()
        
    def forward(self, x, masks, step):
                
        rnn_out = self.RNN(x)
        last_note_date_hs = get_last_note_date(rnn_out[0],masks)
        fc1_out = self.fc1(last_note_date_hs)
        fc1_out = self.relu(fc1_out)
        dp_out = self.dropout(fc1_out)
        #fc2_out = self.fc2(dp_out)
        #out = self.sig(fc2_out).flatten()

        return dp_out

In [ ]:
def get_last_note_date(hidden_states, masks):   
    #last_visit = ((masks.sum(axis = 2) > 0).sum(axis = 1) - 1).unsqueeze(-1)
    #if(step == 134):
    #print(masks)
    #print(hidden_states.shape)
    last_visit = masks.expand(-1,hidden_states.shape[2]).unsqueeze(1)
    
    out = torch.gather(hidden_states,dim = 1,index = last_visit)[:,-1,:]
    return out

In [ ]:
def conv_output_volume(W, K, S, P):    
    return  (((W-K+2*P)//S)+1)